# Question B2 (10 marks)
In Question B1, we used the Category Embedding model. This creates a feedforward neural network in which the categorical features get learnable embeddings. In this question, we will make use of a library called Pytorch-WideDeep. This library makes it easy to work with multimodal deep-learning problems combining images, text, and tables. We will just be utilizing the deeptabular component of this library through the TabMlp network:

In [1]:
pip install pytorch-widedeep

  Using cached pytorch_widedeep-1.6.3-py3-none-any.whl (21.9 MB)
  Using cached sentence_transformers-3.2.0-py3-none-any.whl (255 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 KB 748.3 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 9.4 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 7.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 KB 5.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.8/683.8 KB 6.8 MB/s eta 0:00:00a 0:00:01
  Using cached imutils-0.5.4.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 4.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.9 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 8.2 

In [1]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep import Trainer
from pytorch_widedeep.metrics import R2Score

1.Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from the year 2020 and before as training data, and entries from 2021 and after as the test data.

In [2]:
df = pd.read_csv('hdb_price_prediction.csv')

# YOUR CODE HERE
# Training Data
df_train = df[df['year'] <= 2020].copy()
# Testing Data
df_test = df[df['year'] >= 2021].copy()

# Dropping Unncessary Columns
df_train.drop(columns=['year','full_address','nearest_stn'], inplace=True)
df_test.drop(columns=['year','full_address','nearest_stn'], inplace=True)

print("Training Data:", df_train.shape)
print("Testing Data:", df_test.shape)

Training Data: (87370, 11)
Testing Data: (72183, 11)


In [3]:
df.head()

,month,year,town,full_address,nearest_stn,dist_to_nearest_stn,dist_to_dhoby,degree_centrality,eigenvector_centrality,flat_model_type,remaining_lease_years,floor_area_sqm,storey_range,resale_price
0,1,2017,ANG MO KIO,406 ANG MO KIO AVENUE 10,Ang Mo Kio,1.007264,7.006044,0.016807,0.006243,"2 ROOM, Improved",61.333333,44.0,10 TO 12,232000.0
1,1,2017,ANG MO KIO,108 ANG MO KIO AVENUE 4,Ang Mo Kio,1.271389,7.983837,0.016807,0.006243,"3 ROOM, New Generation",60.583333,67.0,01 TO 03,250000.0
2,1,2017,ANG MO KIO,602 ANG MO KIO AVENUE 5,Yio Chu Kang,1.069743,9.090700,0.016807,0.002459,"3 ROOM, New Generation",62.416667,67.0,01 TO 03,262000.0
3,1,2017,ANG MO KIO,465 ANG MO KIO AVENUE 10,Ang Mo Kio,0.946890,7.519889,0.016807,0.006243,"3 ROOM, New Generation",62.083333,68.0,04 TO 06,265000.0
4,1,2017,ANG MO KIO,601 ANG MO KIO AVENUE 5,Yio Chu Kang,1.092551,9.130489,0.016807,0.002459,"3 ROOM, New Generation",62.416667,67.0,01 TO 03,265000.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159553 entries, 0 to 159552
Data columns (total 14 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   month                   159553 non-null  int64  
 1   year                    159553 non-null  int64  
 2   town                    159553 non-null  object 
 3   full_address            159553 non-null  object 
 4   nearest_stn             159553 non-null  object 
 5   dist_to_nearest_stn     159553 non-null  float64
 6   dist_to_dhoby           159553 non-null  float64
 7   degree_centrality       159553 non-null  float64
 8   eigenvector_centrality  159553 non-null  float64
 9   flat_model_type         159553 non-null  object 
 10  remaining_lease_years   159553 non-null  float64
 11  floor_area_sqm          159553 non-null  float64
 12  storey_range            159553 non-null  object 
 13  resale_price            159553 non-null  float64
dtypes: float64(7), int64

2.Refer to the documentation of Pytorch-WideDeep and perform the following tasks:
https://pytorch-widedeep.readthedocs.io/en/latest/index.html
* Use [**TabPreprocessor**](https://pytorch-widedeep.readthedocs.io/en/latest/examples/01_preprocessors_and_utils.html#2-tabpreprocessor) to create the deeptabular component using the continuous
features and the categorical features. Use this component to transform the training dataset.
* Create the [**TabMlp**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/model_components.html#pytorch_widedeep.models.tabular.mlp.tab_mlp.TabMlp) model with 2 linear layers in the MLP, with 200 and 100 neurons respectively.
* Create a [**Trainer**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/trainer.html#pytorch_widedeep.training.Trainer) for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. Train the model for 100 epochs using this trainer, keeping a batch size of 64. (Note: set the *num_workers* parameter to 0.)

In [5]:
# YOUR CODE & RESULT HERE
cat_embed_cols = [
    ('month', len(np.unique(df['month']))),
    ('town', len(np.unique(df['town']))),
    ('flat_model_type', len(np.unique(df['flat_model_type']))),
    ('storey_range', len(np.unique(df['storey_range']))),
]


continuous_cols = ['dist_to_nearest_stn','dist_to_dhoby','degree_centrality','eigenvector_centrality',
                 'remaining_lease_years','floor_area_sqm']

In [6]:
tab_preprocessor = TabPreprocessor(
    cat_embed_cols = cat_embed_cols, continuous_cols = continuous_cols
)

# Scaled Training Data
X_tab = tab_preprocessor.fit_transform(df_train)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pytorch_widedeep/preprocessing/tab_preprocessor.py:360: UserWarning: Continuous columns will not be normalised
  warnings.warn("Continuous columns will not be normalised")


In [7]:
# Creating the TabMlp Model
model = TabMlp(tab_preprocessor.column_idx, 
                cat_embed_input = tab_preprocessor.cat_embed_input, 
                cat_embed_dropout = 0.1,
                continuous_cols = continuous_cols,
                mlp_hidden_dims = [200, 100])

wide_deep = WideDeep(deeptabular = model) 

In [9]:
# Creating the Trainer 
trainer = Trainer(model = wide_deep,
                  objective = "regression",
                  lr_scheduler_step = False,  
                  num_workers = 0,  
                  metrics = [R2Score])

In [10]:
# Training Model with 100 Epochs with Batch Size 64
trainer.fit(X_tab = X_tab, 
            target = df_train['resale_price'].values, 
            n_epochs = 100, 
            batch_size = 64)

epoch 100: 100%|██████████| 1366/1366 [00:04<00:00, 312.93it/s, loss=2.04e+9, metrics={'r2': 0.914}] 


3.Report the test RMSE and the test R2 value that you obtained.

In [11]:
# YOUR CODE & RESULT HERE
X_test = tab_preprocessor.transform(df_test)

# Make predictions on the test dataset
y_pred = trainer.predict(X_tab = X_test)

predict: 100%|██████████| 1128/1128 [00:01<00:00, 915.95it/s]


In [12]:
y_pred

array([201287.66, 211353.58, 288670.  , ..., 590245.3 , 520264.7 ,
       540961.8 ], dtype=float32)

In [13]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

y_true = df_test['resale_price']

r2 = r2_score(y_true, y_pred)
print("R^2 Score:", r2)
mse = mean_squared_error(y_true, y_pred)
print("Root Mean Squared Error (RMSE):", np.sqrt(mse))

R^2 Score: 0.6359610250405704
Root Mean Squared Error (RMSE): 102076.24282375441
